# Reverse Cohorting in Prisons
----

In this notebook, we aim to look into the effects of reverse cohorting (the quarentining of incoming prisoners for some set time, before introduction into wider prison population) has on the probability of disease importation. This work stems from the work done previously looking into effectiveness of border screening travellers for COVID-19 at airports, and the underlying principles remain laregly unchanged.

In the previous work, we looked at the likelihood that individuals infected with COVID-19 at "some point" before boarding their flight, would not be symptomatic by the time they attempted to cross the border into their destination country. We assumed that screening could only detect infection in symptomatic individuals, and thus success rate was dependent on the flight time (allowing more time for disease to fully incubate). 

It is then clear that we may draw parrallels to the work being ask here then; interpretting the time spent in transit in the previous work, as the time spent in quarentine before being released into the prison population.

In [ ]:
import numpy as np
import pandas as pd
import numpy.random as rand
import scipy.stats as stats
import matplotlib.pyplot as plt
import sira

We now define the distribution to be used for randomly allocating each of our modelled incoming individuals an incubation time 

In [ ]:
covid_inc_dist = stats.lognorm

covid_inc_params = {'s':0.47238, 'scale': np.exp(1.6112)}

In attempts to answer the question posed by HMPPS, we consider a range of scenarios modelled below; each of these presumes the use of a test that is either 50%, 60%, 70% 80%, 90% or 100% effective a detecting infection in symptomatic individuals. For each of these scenarios, a test is administered on both arrival into and exit from the reverse cohorting unit. As such, where a non-perfect test is being used (less than 100% effective) there is a chance that symptomatic individuals may go undetected and proceed to further stages of the model.

In [ ]:
sensitivity = [i/10 for i in range(5,11)]
result_dict1 = {}
for sens in sensitivity:
    detect_rates = []
    for i in [5, 7, 10, 14]:
        d_prob = sira.border_screening(num_people = 100000,
                              flight_dist = rand.uniform,
                              exp_dist = rand.uniform,
                              inc_dist = rand.lognormal, 
                              time_scale = 'days',
                              entry_scr_sens = sens,
                              exit_scr_sens = sens,
                              reporting=True,
                              **{
                                 'exp':{'low':0, 'high':14*24},
                                 'inc':{'sigma':0.47238, 'mean': 1.6112},
                                 'flight':{'low':((i*24)), 'high':((i*24)+0.00001)}
                                 }
                          )
        detect_rates.append(d_prob)
    result_dict1[sens] = detect_rates

In [ ]:
df1 = pd.DataFrame.from_dict(result_dict1, orient='index', columns=[5,7,10,14])
df1